# Random Forest Regressor 

## Mục tiêu
- Xây dựng mô hình Random Forest Regressor để dự đoán tuổi thọ trung bình
- Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`
- Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation
- Đánh giá mô hình trên tập train
- Lưu mô hình đã huấn luyện 


## Giới thiệu

Là tập hợp nhiều cây quyết định độc lập (bagging ensemble).

### **Ưu điểm:**

- Giảm overfitting so với Decision Tree.

- Ổn định, mạnh mẽ.

- Tự động đánh giá tầm quan trọng của biến.

- Xử lý tốt dữ liệu lớn.

### **Nhược điểm:**

- Không dễ diễn giải như 1 cây duy nhất.

- Cần nhiều tài nguyên tính toán.

## Bước 1 - Import các thư viện cần thiết

In [1]:
# Thư viện cơ bản
import numpy as np
import pandas as pd
import os

# Thư viện sklearn cho mô hình và đánh giá
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## Bước 2 - Đọc dữ liệu đã tiền xử lý

Đọc dữ liệu từ các file CSV đã được tiền xử lý và chia sẵn thành train/validation/test.

In [2]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập train (đã được chuẩn hóa):")
train_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (3255, 15)

5 dòng đầu tiên của tập train (đã được chuẩn hóa):


,country_name,country_code,year,population,poverty_ratio,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,slum_population,labor_force
0,Romania,ROU,2019,-0.100387,-0.593216,-1.101137,75.607317,-0.107887,0.079336,1.142611,0.621932,0.746494,-0.047183,-0.857182,-0.871164
1,Mauritius,MUS,2006,-0.235882,-0.685495,-0.500838,72.432195,-0.405891,0.241659,0.204355,0.586683,0.718065,-0.221970,1.767314,-0.189257
2,Angola,AGO,2008,-0.083899,0.283435,1.500859,55.281000,-0.470953,1.327705,-0.839543,-1.545881,-2.268636,-0.432669,0.305989,1.598947
3,Albania,ALB,2001,-0.222240,-0.619770,-1.349940,75.083000,-0.584364,0.930412,-0.835544,0.600783,-0.024271,-0.404910,0.108810,-0.104497
4,Central African Republic,CAF,2021,-0.206911,2.567341,0.236338,40.279000,-0.618434,-0.427187,-1.336901,-2.349558,-3.008624,-0.529029,1.729562,1.172890


## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`.

In [3]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

THÔNG TIN CÁC TẬP DỮ LIỆU
Số lượng đặc trưng: 12

Các đặc trưng được sử dụng:
  1. year
  2. population
  3. poverty_ratio
  4. pop_growth
  5. gdp_per_capita
  6. gdp_growth
  7. sanitation
  8. electricity
  9. water_access
  10. co2_emissions
  11. slum_population
  12. labor_force

Kích thước X_train: (3255, 12)
Kích thước y_train: (3255,)


## Bước 4 - Xây dựng và huấn luyện mô hình Random Forest 

### 4.1. Sử dụng RandomizedSearchCV với 5-Fold Cross-Validation để tìm kiếm siêu tham số tối ưu cho mô hình Random Forest.

**Các siêu tham số cần tối ưu:**
- **n_estimators:** Số lượng cây trong rừng
- **max_depth:** Độ sâu tối đa của mỗi cây
- **min_samples_split:** Số mẫu tối thiểu để chia một nút
- **min_samples_leaf:** Số mẫu tối thiểu ở một nút lá
- **max_features:** Số lượng đặc trưng tối đa xem xét khi chia
- **bootstrap:** Có sử dụng bootstrap sampling hay không

In [4]:
# Định nghĩa không gian siêu tham số để tìm kiếm
param_grid = {
    'n_estimators': [100, 200, 300, 500, 800],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

print(f"Số lượng tổ hợp tham số: {np.prod([len(v) for v in param_grid.values()])}")
print("Phương pháp: RandomizedSearchCV với 5-Fold Cross-Validation")
print("Metric đánh giá: Negative RMSE (neg_root_mean_squared_error)")
print("\nĐang thực hiện tìm kiếm...")

# Tạo RandomizedSearchCV
# Do số lượng siêu tham số cần tìm là rất lớn nên ta chỉ áp dụng RandomizedSearchCV với n_iter=30
grid_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_grid,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    n_iter=30,
    random_state=42
)

# Thực hiện tìm kiếm
grid_search.fit(X_train, y_train)

Số lượng tổ hợp tham số: 2400
Phương pháp: RandomizedSearchCV với 5-Fold Cross-Validation
Metric đánh giá: Negative RMSE (neg_root_mean_squared_error)

Đang thực hiện tìm kiếm...
Fitting 5 folds for each of 30 candidates, totalling 150 fits


,estimator,RandomForestR...ndom_state=42)
,param_distributions,"{'bootstrap': [True, False], 'max_depth': [None, 10, ...], 'max_features': ['sqrt', 'log2', ...], 'min_samples_leaf': [1, 2, ...], ...}"
,n_iter,30
,scoring,'neg_root_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


### 4.2. Hiển thị siêu tham số tối ưu và đánh giá hiệu suất của mô hình tốt nhất.

In [5]:
# Lấy mô hình tốt nhất và tham số tối ưu
best_rf = grid_search.best_estimator_
best_params = grid_search.best_params_

print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params.items():
    print(f"  {param:20s}: {value}")

# Thông tin về mô hình tối ưu
print(f"\nSố lượng cây: {best_rf.n_estimators}")
print(f"Số đặc trưng xem xét mỗi lần chia: {best_rf.max_features}")
print(f"Bootstrap sampling: {best_rf.bootstrap}")

SIÊU THAM SỐ TỐI ƯU
  n_estimators        : 800
  min_samples_split   : 5
  min_samples_leaf    : 1
  max_features        : log2
  max_depth           : None
  bootstrap           : False

Số lượng cây: 800
Số đặc trưng xem xét mỗi lần chia: log2
Bootstrap sampling: False


### 4.3. Đánh giá mô hình trên tập train

In [6]:
# Dự đoán trên tập train
y_pred = best_rf.predict(X_train)

# Tính toán các metrics cho tập train
train_mae = mean_absolute_error(y_train, y_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred))
train_r2 = r2_score(y_train, y_pred)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: Random Forest:")
print(f"RMSE loss: {train_rmse:.3f}")
print(f"MAE loss: {train_mae:.3f}")
print(f"R2 loss: {train_r2:.3f}")

KẾT QUẢ MÔ HÌNH TỐI ƯU
Model: Random Forest:
RMSE loss: 0.292
MAE loss: 0.140
R2 loss: 0.999


### 4.4. Lưu mô hình 

In [7]:
import joblib
import os

# Tạo thư mục lưu model nếu chưa có
os.makedirs('../model/4_random_forest', exist_ok=True)

# Lưu mô hình
joblib.dump(best_rf, "../model/4_random_forest/random_forest.pkl.gz", compress=('gzip', 6))

['../model/4_random_forest/random_forest.pkl.gz']

## Kết luận

### Tổng kết:
1. Mô hình Random Forest đã được xây dựng và tối ưu hóa thành công
2. RandomizedSearchCV với 5-Fold CV đã tìm được bộ siêu tham số tối ưu
3. Mô hình được đánh giá dựa trên 3 metrics: MAE, RMSE và R2